## モデルの作成～コンパイル～学習

In [ ]:
# import
import tensorflow as tf

# create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1]),
    tf.keras.layers.Flatten(input_shape=(28, 28)),
])
# compile model
model.compile(optimizer='sgd', loss='mean_squared_error')
# train model
model.fit("<train_data>", "<train_labels>", epochs=500)
# predict
print(model.predict([10.0]))
# evaluate
print(model.evaluate("<test_data>", "<test_labels>"))

#### layers
最初の層には`input_shape` (list or tupple) を指定する必要がある
* `tf.keras.layers.Dense(units, activation)` : 全結合層
* `tf.keras.layers.Flatten()` : フラット化層
* `tf.keras.layers.Conv2D(filters, kernel_size, activation)` : 畳み込み層
* `tf.keras.layers.MaxPooling2D(pool_size)` : プーリング層

#### activation
* `tf.keras.activations.relu` : ReLU
* `tf.keras.activations.sigmoid` : sigmoid
* `tf.keras.activations.softmax` : softmax

#### optimizer
* `tf.keras.optimizers.Adam(learning_rate)` : Adamオプティマイザ
* `tf.keras.optimizers.SGD(learning_rate)` : SGDオプティマイザ
* `tf.keras.optimizers.RMSprop(learning_rate)` : RMSpropオプティマイザ
* `tf.keras.optimizers.Adagrad(learning_rate)` : Adagradオプティマイザ
* `tf.keras.optimizers.Adadelta(learning_rate)` : Adadeltaオプティマイザ

#### loss
* `tf.keras.losses.mean_squared_error` : MSE
* `tf.keras.losses.binary_crossentropy` : binary_crossentropy
* `tf.keras.losses.categorical_crossentropy` : categorical_crossentropy

#### metrics
* `tf.keras.metrics.Accuracy` : accuracy

## Callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        threshold = 0.90
        if(logs.get("accuracy") >= threshold):
            print(f"Reached {threshold * 100}% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

model.fit("<train_data>", "<train_labels>", epochs=100, callbacks=[callbacks])

## Datasetのロード

In [ ]:
# fashion mnist
# train_images : (60000, 28, 28), 28x28のグレースケール画像
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# preprocess
train_images = train_images / 255.0
test_images = test_images / 255.0